<a href="https://colab.research.google.com/github/BlackpearlJack/BlackpearlJack.github.io/blob/main/Weather_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

# Define coordinates and city names
locations = [
    {"city": "Nairobi", "latitude": -1.2833, "longitude": 36.8167},
    {"city": "Kisumu", "latitude": -0.1022, "longitude": 34.7617},
    {"city": "Nakuru", "latitude": -0.3072, "longitude": 36.0722},
    {"city": "Mombasa", "latitude": -4.0547, "longitude": 39.6636},
]

# Base URL for Open-Meteo API
url = "https://api.open-meteo.com/v1/forecast"

# Function to fetch weather data
def fetch_weather_data():
    results = []
    for loc in locations:
        params = {
            "latitude": loc["latitude"],
            "longitude": loc["longitude"],
            "current_weather": True,  # Correct key for current weather data
        }
        response = requests.get(url, params=params)  # Direct request
        if response.status_code == 200:
            data = response.json()
            current_weather = data.get("current_weather", {})

            # Convert time to Kenyan time (GMT+3)
            utc_time = current_weather.get("time")
            if utc_time:
                # year/month/day hour:minute:second
                kenyan_time = datetime.fromisoformat(utc_time) + timedelta(hours=3)
                formatted_time = kenyan_time.strftime('%Y-%m-%d %H:%M:%S')
            else:
                formatted_time = None

            results.append({
                "City": loc["city"],
                "Temperature_2m": current_weather.get("temperature"),
                "Precipitation": current_weather.get("precipitation", 0),  # Default to 0 if missing
                "Wind_Speed_10m": current_weather.get("windspeed"),
                "Time (Kenyan)": formatted_time,
            })
        else:
            print(f"Failed to fetch data for {loc['city']} (HTTP {response.status_code})")
    return results

# Run the loop for 12 hours and store data
data_list = []
start_time = time.time()
end_time = start_time + 11 * 60  # 12 hours in seconds

print("Starting weather data collection for 12 hours...")

try:
    while time.time() < end_time:
        print("Fetching latest weather data...")
        weather_data = fetch_weather_data()
        data_list.extend(weather_data)  # Append data to the list
        df = pd.DataFrame(weather_data)
        print(df)
        print(f"Updated at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("\nWaiting for 10 minutes before next update...\n")
        time.sleep(600)  # Wait for 600 seconds (10 minutes)
except KeyboardInterrupt:
    print("Weather update stopped manually.")

# Save collected data to CSV
output_file = "hello.csv"
# .txt, .csv, .xlsx, .json, database(.sql), etc.
all_data_df = pd.DataFrame(data_list)
all_data_df.to_csv(output_file, index=False)
print(f"\nWeather data collection completed. Data saved to '{output_file}'.")


Starting weather data collection for 12 hours...
Fetching latest weather data...
      City  Temperature_2m  Precipitation  Wind_Speed_10m        Time (Kenyan)
0  Nairobi            24.7              0            10.1  2024-11-30 12:00:00
1   Kisumu            25.0              0             6.6  2024-11-30 12:00:00
2   Nakuru            22.4              0            16.2  2024-11-30 12:00:00
3  Mombasa            34.2              0            12.7  2024-11-30 12:00:00
Updated at: 2024-11-30 09:12:09

Waiting for 10 minutes before next update...

